In [1]:
import boto3
import json
import configparser
from aws_policy.policy import s3_access, glue_access

In [2]:
config = configparser.ConfigParser()
config.read_file(open('/Users/andrekrasinski/Documents/GitHub/DataEngineerAWS/dwh.cfg'))

KEY    = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')
REGION = config.get('AWS','REGION')
TOKEN  = config.get('AWS','TOKEN')

EC2_IAM_ROLE_NAME = config.get("EC2", "EC2_IAM_ROLE_NAME")

EC2_VPC_ID = config.get("EC2", "EC2_VPC_ID")
EC2_ROUTE_TABLE_ID = config.get("EC2", "EC2_ROUTE_TABLE_ID")





In [4]:
#Criando s3 resource -- Resource provides a higher-level, object-oriented interface
s3 = boto3.resource(
    's3', 
    region_name=REGION, 
    aws_access_key_id=KEY, 
    aws_secret_access_key=SECRET,
    aws_session_token=TOKEN
)

#Criando ec2 client -- Client provides low-level access to AWS service operations
ec2 = boto3.client(
    'ec2', 
    region_name=REGION, 
    aws_access_key_id=KEY, 
    aws_secret_access_key=SECRET, 
    aws_session_token=TOKEN
)

#Criando IAM client
iam = boto3.client(
    'iam', 
    region_name=REGION, 
    aws_access_key_id=KEY, 
    aws_secret_access_key=SECRET, 
    aws_session_token=TOKEN
)


In [ ]:
#Criando bucket no s3

createBucket = s3.create_bucket(
    Bucket="krasinski-udacity-1",
    CreateBucketConfiguration={'LocationConstraint': 'us-west-2'}
)

In [ ]:
#Gerando a descrição da Virtual Private Cloud (VPC) para pegar o VPC ID
VpcDescribeEndpoint = ec2.describe_vpcs()
print(VpcDescribeEndpoint)

In [ ]:
#Gerando a descrição da Route Tables para pegar o RouteTableID para configurar com VPC Gateway. 
describeRouteTables= ec2.describe_route_tables()
print(describeRouteTables)

In [ ]:
createVpcEndpoint = ec2.create_vpc_endpoint(
    VpcId=EC2_VPC_ID,
    ServiceName=f'com.amazonaws.{REGION}.s3',
    RouteTableIds=[EC2_ROUTE_TABLE_ID]        
)

print(createVpcEndpoint)

In [ ]:
try:
    print("1.1 Creating a new IAM Role")
    dwhRole = iam.create_role(
        RoleName=EC2_IAM_ROLE_NAME,
        Description = "Allows Glue service to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps({
            'Statement': [{'Action': 'sts:AssumeRole',
                           'Effect': 'Allow',
                           'Principal': {'Service': 'glue.amazonaws.com'}}],
            'Version': '2012-10-17'
        })
    )
except Exception as e:
    print(e)

In [ ]:
# Adicionado ao Role as politicas de acesso ao S3 
putRolePolicyS3 = iam.put_role_policy(
    RoleName=EC2_IAM_ROLE_NAME,
    PolicyName='s3Access',
    PolicyDocument=json.dumps(s3_access)
)

print(putRolePolicyS3)

In [ ]:
# Adicionado ao Role as politicas de acesso ao Glue 
putRolePolicyS3 = iam.put_role_policy(
    RoleName=EC2_IAM_ROLE_NAME,
    PolicyName='GlueAccess',
    PolicyDocument=json.dumps(glue_access)
)

print(putRolePolicyS3)